# 2NN-MEAM parameter check

This Notebook is designed to compare the parameters in the hosted 2NN-MEAM parameter files with 

In [1]:
from pathlib import Path

import pandas as pd

import numpy as np

import meam2nn

## Define function for comparing LAMMPS parameter files

__NOTE:__ This function works for the parameter files not the library files.  Needs to be modified in the future or a new function needs to be created for the library file parameters. 

In [2]:
def compare_parameters(params1, params2, ignore_missing=None, ignore_different=None, rtol=1e-05, atol=1e-08):
    """
    Compares MEAM parameters from two parameter files and identifies the parameters that
    are missing from one or the other files and the paramters that have different values.
    
    Parameters
    ----------
    params1 : dict
        The parameter file parameters from one of the files to compare.
    params2 : dict
        The parameter file parameters from one of the files to compare.
    ignore_missing : list, optional
        Parameter keys to ignore if missing from one of the files.
    ignore_different : list, optional
        Parameter keys to ignore if the value is different between the files.
    rtol : float, optional
        The relative tolerance to use in comparing numerical values.  Default value is 1e-5.
    atol : float, optional
        The absolute tolerance to use in comparing numerical values.  Default value is 1e-8.
    """
    
    missing1 = []
    missing2 = []
    different = []
    if ignore_missing is None:
        ignore_missing = []
    if ignore_different is None:
        ignore_different = []
    
    for key in params1:
        if key not in params2:
            if key not in ignore_missing:
                missing2.append(key)
        else:
            try:
                if not np.isclose(params1[key], params2[key], rtol=rtol, atol=atol) and key not in ignore_different:
                    different.append(key)
            except:
                if params1[key] != params2[key]:
                    different.append(key)                
        
    for key in params2:
        if key not in params1:
            if key not in ignore_missing:
                missing1.append(key)
    
    if len(missing1) > 0:
        print('First parameter set is missing:')
        for key in missing1:
            print('-', key)
    
    if len(missing2) > 0:
        print('Second parameter set is missing:')
        for key in missing2:
            print('-', key)
            
    if len(different) > 0:
        print('Different values are:')
        for key in different:
            print('-', key)
            print('  *', params1[key])
            print('  *', params2[key])
            
    if len(missing1) == 0 and len(missing2) == 0 and len(different) == 0:
        print('All values match!')

## Compare files to database parameters


### Parameter database

The database parameters are stored in the three .csv files located in meam2nn/parameters. The format of those files is similar to the sections of the EAM_DB.TDB file used by KISSMD but expanded with new fields to handle more options and less limits on value precisions.

Notable differences with EAM_DB.TDB:

- Elements and model symbols are separate fields.
- First publication DOIs are listed for each interaction where found.
- Either B, B\*, or alpha can be given for the bulk modulus/alpha parameters.  B is bulk mod in eV/A^3, B* is bulk mod in 10^12 dyn/cm^2.  Only one should be given and conversion to others is done automatically.
- d+ and d- are given separately.
- nn2 and Rcut values are included.
- All L12AB3 alloy listings have been converted into L12A3B listings.
- For alloy Cmin values, default averages are used if parameter file is missing rather than set to 0.  All Cmax values are listed even if 2.8.
    
Things still to do or consider:

- Separating elements and symbols allows for multiple parameterizations of the same element to be included.  However, for alloys and ternaries, only one parameterization for a given symbols set is loaded. This means old archival parameterizations require either separate csv files or another column is needed to allow for unique model identification.
- Consistent handling of default values is needed.  Right now, d values are handled like KISSMD does but Cmin/Cmax defaults are handled differently (see above).  Also, there are multiple Cmax defaults for alloy and ternary interactions: 2.8, averaging, ...  

In [3]:
# Select the parameter file location to compare to parameter values
location = 'lammps'
#location = 'csme'

# Load generator class and interactions.csv
db = meam2nn.MEAM2NN()
interactions = pd.read_csv('interactions.csv')

# list fields to ignore if not found in one or the other parameter sets
ignore_missing = ['mixture_ref_t', 'emb_lin_neg', 'bkgd_dyn', 'alpha(1,1)', 'alpha(2,2)', 'alpha(3,3)',
                  'Cmin(2,1,3)', 'Cmin(3,1,2)', 'Cmin(3,2,1)', 'Cmax(2,1,3)', 'Cmax(3,1,2)', 'Cmax(3,2,1)']

# List fields to ignore if values differ between the parameter sets
ignore_different = ['rc']

# List fields to ignore for single element potentials only
ignore_element = ['ialloy', 'rho0(1)']

# Set parameters based on location value
if location == 'lammps': 
    root = Path('LAMMPS')
    paramkey = 'parameter file LAMMPS'
elif location == 'csme':
    root = Path('cmse_postech_ac_kr')
    paramkey = 'parameter file csme'

# Loop over rows in interactions.csv
for i in interactions.index:
    row = interactions.loc[i]
    symbols = row.interaction.split('-')
    print(row.interaction)
     
    try:
        # Generate parameter file from database
        paramfile = db.lammps_parameter(symbols)
    except:
        print('No matching database parameters found')
        print()
        continue
    else:
        # Parse the generated parameter file
        params1 = meam2nn.parse_lammps_parameter_file(paramfile)
    
    try:
        # Identify parameter file's location
        paramfile = Path(root, row[paramkey])
        assert paramfile.is_file()
    except:
        print('No parameter file found')
        print()
        continue        
    else:
        # Load and parse the parameter file
        params2 = meam2nn.parse_lammps_parameter_file(paramfile)
    
    if len(symbols) == 1:
        # Compare parameters for single element models
        compare_parameters(params1, params2, ignore_missing=ignore_missing+ignore_element,
                           ignore_different=ignore_different+ignore_element)
        print()
    
    else:
        # Compare parameters for multi-element models
        compare_parameters(params1, params2, ignore_missing=ignore_missing, ignore_different=ignore_different)
        print()

Cr
All values match!

Fe
All values match!

Mo
All values match!

Nb
All values match!

Ta
All values match!

V
All values match!

W
All values match!

Fe-Cr
All values match!

Ag
All values match!

Al
All values match!

Au
All values match!

Cu
All values match!

Ni
All values match!

Pb
All values match!

Pd
All values match!

Pt
All values match!

Ni-W
All values match!

Ni-Cu
All values match!

C
All values match!

Fe-Cu
All values match!

Pt-Fe
All values match!

Ti
All values match!

Zr
All values match!

Fe-C
All values match!

Fe-N
All values match!

Ti-Cu
All values match!

Fe-H
All values match!

Si
All values match!

Al-Ni
All values match!

In
All values match!

Ge
All values match!

Zr-Cu
All values match!

Ti-C
All values match!

Ti-N
All values match!

Nb-Fe
All values match!

Fe-Ti
All values match!

Ga-In
Different values are:
- Ec(1,2)
  * 2.8405
  * 4.06
- alpha(1,2)
  * 4.63256713599982
  * 3.87486077
- attrac(1,2)
  * 0.0735
  * 0.074
- repuls(1,2)
  * 0.0735
  * 0

## Check database parameters for an interaction

In [5]:
db = meam2nn.MEAM2NN()
db.element('Al')

El                                  Al
Sym                                 Al
DOI         10.1103/PhysRevB.68.144112
Ref.St                          FCC_A1
mass                            26.982
Ec                                3.36
Re                                2.86
B                               0.4955
B*                            0.793879
alpha                           4.6856
A                                 1.16
beta(0)                            3.2
beta(1)                            2.6
beta(2)                              6
beta(3)                            2.6
t(1)                              3.05
t(2)                              0.51
t(3)                              7.75
Rho_zero                             1
Cmin                              0.49
Cmax                               2.8
d+                                0.05
d-                                0.05
nn2                                  1
Rcut                               4.5
lat                      

In [6]:
db = meam2nn.MEAM2NN()
db.alloy(['Al', 'Co'])

El1                                    Al
El2                                    Co
Sym1                                   Al
Sym2                                   Co
DOI         10.1016/j.calphad.2012.04.001
Ref.St                             BCC_B2
delta_Ec                           -0.565
Re                                  2.476
B                                  1.0112
B*                                1.62012
alpha                             4.88849
d+                                  0.025
d-                                  0.025
Cmin_iji                              1.1
Cmin_jij                             0.49
Cmin_iij                             0.49
Cmin_ijj                             0.49
Cmax_iji                              2.8
Cmax_jij                                2
Cmax_iij                              2.8
Cmax_ijj                             1.44
Rcut                                  4.5
Au                                     DW
lat                               

In [7]:
db = meam2nn.MEAM2NN()
db.ternary(['Ni', 'Al', 'Co'])

El1                                    Ni
El2                                    Al
El3                                    Co
Sym1                                   Ni
Sym2                                   Al
Sym3                                   Co
DOI         10.1088/0965-0393/23/5/055004
Cmax_ikj                             1.44
Cmax_ijk                             2.38
Cmax_jik                             1.44
Cmin_ikj                             0.56
Cmin_ijk                             0.97
Cmin_jik                             0.56
Rcut                                  4.5
AU                                    NaN
Name: 10, dtype: object

## Ga-In-N still not right?

In [43]:
E_Ga = 2.9
E_In = 2.51
E_NN = 4.88

dE_GaIn = -.1355
dE_GaNN = 0.56
dE_InNN = 0.1650

print(0.5 * E_Ga + 0.5* E_In - dE_GaIn)
print(0.5 * E_Ga + 0.5* E_NN - dE_GaNN)
print(0.5 * E_In + 0.5* E_NN - dE_InNN)

2.8405
3.3299999999999996
3.53


In [44]:
0.5 * (E_Ga + E_In) + 0.1355

2.8405

In [45]:
0.5 * (E_Ga + E_NN) - 0.56

3.3299999999999996

In [46]:
0.5 * (E_In + E_NN) - 0.1650

3.53